In [2]:
import pandas as pd
import re
import nltk
import json
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm

# 第一次运行请取消注释这两行以确保下载需要的资源
# nltk.download("punkt")
# nltk.download("stopwords")
# nltk.download("wordnet")

# 初始化处理器
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'(https?://\S+|www\.\S+)', ' ', text)
    text = text.replace('|', ' ')
    text = re.sub(r'[^a-z\s]', ' ', text)
    tokens = nltk.word_tokenize(text)
    cleaned = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token not in stop_words and len(token) > 2
    ]
    return ' '.join(cleaned)

def flatten_nested_json(raw_data):
    """
    将形如 {"INFP": [...], "INTJ": [...]} 的字典结构打平成一个 list[dict]
    """
    flat_list = []
    for key in raw_data:
        entries = raw_data[key]
        if isinstance(entries, list):
            for item in entries:
                if isinstance(item, dict) and "post" in item and "type" in item:
                    flat_list.append(item)
    return flat_list

def preprocess_nested_json(input_path, output_path_csv=None, output_path_json=None):
    print(f"📥 加载 JSON 数据: {input_path}")

    with open(input_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    flat_data = flatten_nested_json(raw_data)

    if not flat_data:
        raise ValueError("❌ 未找到合法的 {'type': ..., 'post': ...} 数据结构")

    df = pd.DataFrame(flat_data)
    print(f"✅ 扁平化完成，共 {len(df)} 条")

    tqdm.pandas(desc="Cleaning")
    df["posts_cleaned"] = df["post"].astype(str).progress_apply(clean_text)

    # 保存 CSV
    if output_path_csv:
        os.makedirs(os.path.dirname(output_path_csv), exist_ok=True)
        df.to_csv(output_path_csv, index=False)
        print(f"✅ 保存 CSV 至：{output_path_csv}")

    # 可选：保存为 JSON
    # 可选：保存为 JSON
    if output_path_json:
        os.makedirs(os.path.dirname(output_path_json) or ".", exist_ok=True)
        with open(output_path_json, 'w', encoding='utf-8') as f:
            json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
        print(f"✅ 保存 JSON 至：{output_path_json}")




In [3]:
# # 示例用法
# if __name__ == "__main__":
#     input_json = "augmented_data/all_augmented_data_v17.json"
#     output_csv = "augmented_data/cleaned_all_augmented_data_with_original_posts_v17.csv"
#     output_json = "cleaned_all_augmented_data_with_original_posts_v17.json"

#     preprocess_nested_json(input_json, output_path_csv=output_csv, output_path_json=output_json)

In [4]:
import pandas as pd
import re
import nltk
import json
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
def preprocess_nested_json(input_path, output_path_csv=None, output_path_json=None):
    print(f"📥 加载 JSON 数据: {input_path}")

    with open(input_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    # 判断结构：如果是 list[dict]，直接处理；如果是 dict，再 flatten
    if isinstance(raw_data, list):
        flat_data = raw_data
    elif isinstance(raw_data, dict):
        flat_data = flatten_nested_json(raw_data)
    else:
        raise ValueError("❌ 不支持的数据结构类型")

    if not flat_data:
        raise ValueError("❌ 未找到合法的 {'type': ..., 'post': ...} 数据结构")

    df = pd.DataFrame(flat_data)
    print(f"✅ 扁平化完成，共 {len(df)} 条")

    tqdm.pandas(desc="Cleaning")
    df["posts_cleaned"] = df["posts"].astype(str).progress_apply(clean_text)

    if output_path_csv:
        output_dir = os.path.dirname(output_path_csv)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
        df.to_csv(output_path_csv, index=False)
        print(f"✅ 保存 CSV 至：{output_path_csv}")

    if output_path_json:
        output_dir = os.path.dirname(output_path_json)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
        with open(output_path_json, 'w', encoding='utf-8') as f:
            json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
        print(f"✅ 保存 JSON 至：{output_path_json}")

In [5]:
# 示例用法
if __name__ == "__main__":
    input_json = "filtered_processed_comments_300.json"
    output_csv = "filtered_processed_comments__300_cleaned.csv"
    output_json = "filtered_processed_comments_300_cleaned.json"

    preprocess_nested_json(input_json, output_path_csv=output_csv, output_path_json=output_json)

📥 加载 JSON 数据: filtered_processed_comments_300.json
✅ 扁平化完成，共 1261235 条


Cleaning: 100%|██████████| 1261235/1261235 [20:00<00:00, 1050.63it/s]


✅ 保存 CSV 至：filtered_processed_comments__300_cleaned.csv
✅ 保存 JSON 至：filtered_processed_comments_300_cleaned.json


In [3]:
import ijson

filename = 'filtered_processed_comments_cleaned.json'
count = 0
max_items = 5
max_value_length = 100  # 每个字段值显示的最大字符数

with open(filename, 'r', encoding='utf-8') as f:
    parser = ijson.items(f, 'item')

    for obj in parser:
        print(f"\n🔹 第 {count + 1} 个字典：")
        for key, value in obj.items():
            # 字符串值裁剪，其他类型转为字符串后裁剪
            val_str = str(value)
            if len(val_str) > max_value_length:
                val_str = val_str[:max_value_length] + '...'
            print(f"  {key}: {val_str}")
        count += 1
        if count >= max_items:
            break



🔹 第 1 个字典：
  type: INTP
  posts: What languages do you speak?|||That's authoritarianism versus libertarianism, not left versus right....
  posts_cleaned: language speak authoritarianism versus libertarianism left versus right know everyone experience dep...

🔹 第 2 个字典：
  type: ENTP
  posts: It is just arguing semantics. To many on the consumer end, beta and production phase both share the ...
  posts_cleaned: arguing semantics many consumer end beta production phase share fact mainstream release ready intere...

🔹 第 3 个字典：
  type: ENTJ
  posts: Come to Europe if you're interested. In many EU countries doing a PhD is actually a job, for which y...
  posts_cleaned: come europe interested many country phd actually job get paid salary hassle tuition mean bullshit sc...

🔹 第 4 个字典：
  type: INTP
  posts: Exactly, so I won't have to eat a dick, I knew what I was saying.Two favorite goddesses are Artemis ...
  posts_cleaned: exactly eat dick knew saying two favorite goddess artemis athena mea